In [ ]:
import pandas as pd
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install fuzzywuzzy
!pip install jellyfish
!pip install transformers
!pip install python-Levenshtein

In [ ]:
!pip install textdistance

In [ ]:
import pandas as pd
import os

# Define the directory paths and asjust when wanting to change the generated directory
truth_dir = '/content/drive/My Drive/LLM-Data/truth'
generated_dir = '/content/drive/My Drive/LLM-Data/llm-generated/GPT4_Enslaved_MainAgent'

# code to fix how names are included so we can match them

# List of files in the truth directory
truth_files = os.listdir(truth_dir)

# Function to transform name from "First Last" format to "Last_First"
def transform_name(name):
    parts = name.split(" ")
    if len(parts) == 2:
        return f"{parts[1]}_{parts[0]}"  # Reverse the order and join with underscore
    return name

# Process each truth file and compare it against generated files
def process_files(truth_file):
    # Load the truth file without headers
    truth_path = os.path.join(truth_dir, truth_file)
    truth_df = pd.read_csv(truth_path, sep='\t', header=None)

    # Display the truth DataFrame for debugging
    print(f"\nProcessing truth file: {truth_file}")
    print("Truth DataFrame:")
    print(truth_df)

    # Prepare the generated file base name by transforming the truth file name
    generated_base_name = transform_name(truth_file.strip('.tsv'))

    # Iterate over generated files in the generated directory
    for generated_file in os.listdir(generated_dir):
        # Check if the generated_file is a file and matches the base name
        if os.path.isfile(os.path.join(generated_dir, generated_file)) and generated_base_name in generated_file:
            generated_path = os.path.join(generated_dir, generated_file)
            print(f"\nFound matching generated file: {generated_file}")

            # Load the generated file and drop the first row
            generated_df = pd.read_csv(generated_path, sep='\t', header=0)
            generated_df = generated_df.drop(index=0)  # Drop the first row

            # Display the generated DataFrame for debugging
            print("Generated DataFrame before transformation:")
            print(generated_df)

            # Transform the Subject column dynamically
            generated_df['Subject'] = generated_df['Subject'].apply(lambda x: x.replace('_triples', '').replace('_', ' ').strip())

            # Normalize text for matching
            truth_df[1] = truth_df[1].str.lower()  # Assuming Predicate is in the second column
            truth_df[2] = truth_df[2].str.lower()  # Assuming Object is in the third column
            generated_df['Predicate'] = generated_df['Predicate'].str.lower()
            generated_df['Object'] = generated_df['Object'].str.lower()

            # Display the transformed generated DataFrame for debugging
            print("Generated DataFrame after transformation:")
            print(generated_df)

            # Perform similarity metrics matching
            for index, truth_row in truth_df.iterrows():
                match_found = False  # Track if any match is found
                for _, generated_row in generated_df.iterrows():
                    # Print debug information to see what is being compared
                    print(f"Comparing: {truth_row[1]} -> {truth_row[2]} with {generated_row['Predicate']} -> {generated_row['Object']} (Subject: {generated_row['Subject']})")

                    # Compare Predicate, Object pairs
                    if truth_row[1] == generated_row['Predicate'] and truth_row[2] == generated_row['Object']:
                        print(f"Match found for truth row {truth_row[0]} with generated row {generated_row['Subject']}")
                        match_found = True
                if not match_found:
                    print(f"No match found for truth row: {truth_row[0]}, Predicate: {truth_row[1]}, Object: {truth_row[2]}")

# Process each file in the truth directory
for truth_file in truth_files:
    process_files(truth_file)


Streaming output truncated to the last 5000 lines.
Comparing: hassurname -> haralson with fullnameasstring -> jeremiah haralson (Subject: Jeremiah Haralson)
Comparing: hassurname -> haralson with hassurnameasstring -> haralson (Subject: Jeremiah Haralson)
Comparing: hassurname -> haralson with hasfirstnameasstring -> jeremiah (Subject: Jeremiah Haralson)
Comparing: hassurname -> haralson with hasdescription -> born into slavery, self-educated, served in congress, convicted of pension fraud, vanished from historical record (Subject: Jeremiah Haralson)
Comparing: hassurname -> haralson with referstoplaceoforigin -> columbus, georgia (Subject: Jeremiah Haralson)
Comparing: hassurname -> haralson with has_race -> african-american (Subject: Jeremiah Haralson)
Comparing: hassurname -> haralson with hassex -> male (Subject: Jeremiah Haralson)
Comparing: hassurname -> haralson with hasstatusgeneratedevent -> enslavement, self-education, congressional service, conviction for pension fraud (Subj

KeyboardInterrupt: 

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install jellyfish
!pip install scikit-learn
!pip install bert-score


In [ ]:
import os

# code to make sure it finds the matching files, adjust directory when using a different generated one

# Define the directory paths
truth_dir = '/content/drive/My Drive/LLM-Data/truth'
generated_dir = '/content/drive/My Drive/LLM-Data/llm-generated/GPT4_Enslaved_MainAgent'

# Get the list of files in the truth directory
truth_files = os.listdir(truth_dir)

# Function to transform name from "First Last" format to "Last_First"
def transform_name(name):
    parts = name.split(" ")
    if len(parts) == 2:
        return f"{parts[1]}_{parts[0]}"  # Reverse the order and join with underscore
    return name

# List to hold the pairs of matching files
matching_pairs = []

# Iterate over truth files and find corresponding generated files
for truth_file in truth_files:
    if truth_file.endswith('.tsv'):  # Check if the file is a TSV
        # Prepare the generated file base name by transforming the truth file name
        generated_base_name = transform_name(truth_file.strip('.tsv'))

        # Check for generated files in the generated directory
        for generated_file in os.listdir(generated_dir):
            if generated_base_name in generated_file and generated_file.endswith('.tsv'):
                matching_pairs.append((truth_file, generated_file))

# Print the matching pairs
for truth_file, generated_file in matching_pairs:
    print(f"Truth File: {truth_file} <--> Generated File: {generated_file}")


Truth File: William_Lewis.tsv <--> Generated File: William_Lewis_triples.tsv
Truth File: David_Abner.tsv <--> Generated File: David_Abner_triples.tsv
Truth File: William_Harvey_Carney.tsv <--> Generated File: William_Harvey_Carney_triples.tsv
Truth File: Thomas_Sims.tsv <--> Generated File: Thomas_Sims_triples.tsv
Truth File: John_Edward_Bruce.tsv <--> Generated File: John_Edward_Bruce_triples.tsv
Truth File: James_Collins_Johnson.tsv <--> Generated File: James_Collins_Johnson_triples.tsv
Truth File: Stephen_Bishop.tsv <--> Generated File: Stephen_Bishop_(cave_explorer)_triples.tsv
Truth File: John_Horse.tsv <--> Generated File: John_Horse_triples.tsv
Truth File: Josiah_Henson.tsv <--> Generated File: Josiah_Henson_triples.tsv
Truth File: Asa_Coleman.tsv <--> Generated File: Asa_Coleman_triples.tsv
Truth File: Fortune.tsv <--> Generated File: Amos_Fortune_triples.tsv
Truth File: Fortune.tsv <--> Generated File: Fortune_(American_slave)_triples.tsv
Truth File: Fortune.tsv <--> Generated

In [ ]:
import os
import pandas as pd

# Define the generated directory path
generated_dir = '/content/drive/My Drive/LLM-Data/llm-generated/GPT4_Enslaved_notrestrictedToMAgent'

# List to hold processed DataFrames for verification
processed_dataframes = []

# Process each generated file in the directory
for generated_file in os.listdir(generated_dir):
    if generated_file.endswith('.tsv'):
        # Load the generated file
        generated_path = os.path.join(generated_dir, generated_file)
        generated_df = pd.read_csv(generated_path, sep='\t', header=0)

        # Drop the first row
        generated_df = generated_df.drop(index=0)

        # Modify the 'Subject' column by removing underscores and inverting words
        generated_df['Subject'] = generated_df['Subject'].apply(
            lambda x: ' '.join(x.replace('_', ' ').replace("triples", "").split()[::-1])
        )

        # Add the processed DataFrame to the list for later inspection
        processed_dataframes.append((generated_file, generated_df))

        # Print one example processed file (the first one)
        if len(processed_dataframes) == 1:
            print(f"Processed DataFrame from {generated_file}:")
            print(generated_df.head())  # Display the first few rows of the DataFrame

# Optionally, you can save the modified DataFrames if needed
# for file_name, df in processed_dataframes:
#     output_path = os.path.join(generated_dir, f"processed_{file_name}")
#     df.to_csv(output_path, sep='\t', index=False)


Processed DataFrame from Yarrow_Mamout_triples.tsv:
         Subject                Predicate  \
1  Mamout Yarrow  hasPreferredNameVariant   
2  Mamout Yarrow         fullNameAsString   
3  Mamout Yarrow       hasSurnameAsString   
4  Mamout Yarrow     hasFirstnameAsString   
5  Mamout Yarrow           hasDescription   

                                              Object  
1                                      Yarrow Mamout  
2                                      Yarrow Mamout  
3                                             Mamout  
4                                             Yarrow  
5  Formerly enslaved African entrepreneur and pro...  


In [ ]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
import jellyfish
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from bert_score import score as bert_score

# Define the directory paths and adjust when dealing with different generated files
truth_dir = '/content/drive/My Drive/LLM-Data/truth'
generated_dir = '/content/drive/My Drive/LLM-Data/llm-generated/GPT4_Enslaved_MainAgent'
output_dir = '/content/drive/My Drive/LLM-Data/evaluate-test/GPT4_Enslaved_MainAgent'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Transform name from "First Last" format to "Last_First"
def transform_name(name):
    parts = name.split(" ")
    if len(parts) == 2:
        return f"{parts[1]}_{parts[0]}"  # Reverse the order and join with underscore
    return name

# Get the list of files in the truth directory and create matching pairs
truth_files = os.listdir(truth_dir)
matching_pairs = []

for truth_file in truth_files:
    if truth_file.endswith('.tsv'):
        generated_base_name = transform_name(truth_file.strip('.tsv'))

        # Look for generated files matching the base name
        for generated_file in os.listdir(generated_dir):
            if generated_base_name in generated_file and generated_file.endswith('.tsv'):
                matching_pairs.append((truth_file, generated_file))

# Print the matching pairs
for truth_file, generated_file in matching_pairs:
    print(f"Truth File: {truth_file} <--> Generated File: {generated_file}")

# Process the matched pairs and calculate similarity metrics
for truth_file, generated_file in matching_pairs:
    print(f"\nProcessing truth file: {truth_file} with generated file: {generated_file}")

    # Load the truth file
    truth_path = os.path.join(truth_dir, truth_file)
    truth_df = pd.read_csv(truth_path, sep='\t', header=None)
    truth_df[1] = truth_df[1].str.lower()  # Normalize predicates
    truth_df[2] = truth_df[2].str.lower()  # Normalize objects

    # Load the generated file
    generated_path = os.path.join(generated_dir, generated_file)
    generated_df = pd.read_csv(generated_path, sep='\t', header=0).drop(index=0)

    # Modify the Subject column
    generated_df['Subject'] = generated_df['Subject'].apply(
        lambda x: ' '.join(x.replace('_', ' ').replace("triples", "").split()[::-1])
    )

    # Normalize generated predicates and objects
    generated_df['Predicate'] = generated_df['Predicate'].str.lower()
    generated_df['Object'] = generated_df['Object'].str.lower()

    results = []

    # Use a TfidfVectorizer instance outside the loop for efficiency
    vectorizer = TfidfVectorizer()

    # Debugging: Print the number of rows in truth and generated DataFrames
    print(f"Number of truth triples: {len(truth_df)}")
    print(f"Number of generated triples: {len(generated_df)}")

    # Iterate through truth and generated triples for similarity metrics
    for truth_index, truth_row in truth_df.iterrows():
        truth_pred, truth_obj = truth_row[1], truth_row[2]

        for generated_index, generated_row in generated_df.iterrows():
            generated_pred, generated_obj = generated_row['Predicate'], generated_row['Object']

            # Debugging: Print the currently compared triples
            print(f"Comparing Truth Predicate: {truth_pred} with Generated Predicate: {generated_pred}")
            print(f"Comparing Truth Object: {truth_obj} with Generated Object: {generated_obj}")

            # Calculate similarity metrics
            metrics = {
                "Predicate Fuzzy Ratio": fuzz.ratio(truth_pred, generated_pred),
                "Predicate Jaro-Winkler": jellyfish.jaro_winkler_similarity(truth_pred, generated_pred),
                "Object Fuzzy Ratio": fuzz.ratio(truth_obj, generated_obj),
                "Object Jaro-Winkler": jellyfish.jaro_winkler_similarity(truth_obj, generated_obj),
            }

            # Calculate Cosine Similarity using a single vectorizer call
            vectors = vectorizer.fit_transform([truth_pred, generated_pred]).toarray()
            metrics["Predicate Cosine Similarity"] = cosine_similarity(vectors)[0, 1]

            vectors = vectorizer.fit_transform([truth_obj, generated_obj]).toarray()
            metrics["Object Cosine Similarity"] = cosine_similarity(vectors)[0, 1]

            # BERTScore for predicates and objects
            P, R, F1 = bert_score([truth_pred], [generated_pred], lang='en', verbose=False)
            metrics["Predicate BERTScore"] = F1.mean().item()

            P, R, F1 = bert_score([truth_obj], [generated_obj], lang='en', verbose=False)
            metrics["Object BERTScore"] = F1.mean().item()

            # Append results
            results.append({
                "Truth Predicate": truth_pred,
                "Truth Object": truth_obj,
                "Generated Predicate": generated_pred,
                "Generated Object": generated_obj,
                **metrics
            })

    # Save results to a CSV file for this pair
    results_df = pd.DataFrame(results)
    output_file_name = f"{os.path.splitext(generated_file)[0]}_evaluation.csv"
    output_file_path = os.path.join(output_dir, output_file_name)
    results_df.to_csv(output_file_path, index=False)
    print(f"Results saved to: {output_file_path}")


Truth File: William_Lewis.tsv <--> Generated File: William_Lewis_triples.tsv
Truth File: David_Abner.tsv <--> Generated File: David_Abner_triples.tsv
Truth File: William_Harvey_Carney.tsv <--> Generated File: William_Harvey_Carney_triples.tsv
Truth File: Thomas_Sims.tsv <--> Generated File: Thomas_Sims_triples.tsv
Truth File: John_Edward_Bruce.tsv <--> Generated File: John_Edward_Bruce_triples.tsv
Truth File: James_Collins_Johnson.tsv <--> Generated File: James_Collins_Johnson_triples.tsv
Truth File: Stephen_Bishop.tsv <--> Generated File: Stephen_Bishop_(cave_explorer)_triples.tsv
Truth File: John_Horse.tsv <--> Generated File: John_Horse_triples.tsv
Truth File: Josiah_Henson.tsv <--> Generated File: Josiah_Henson_triples.tsv
Truth File: Asa_Coleman.tsv <--> Generated File: Asa_Coleman_triples.tsv
Truth File: Fortune.tsv <--> Generated File: Amos_Fortune_triples.tsv
Truth File: Fortune.tsv <--> Generated File: Fortune_(American_slave)_triples.tsv
Truth File: Fortune.tsv <--> Generated

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Comparing Truth Predicate: hasname with Generated Predicate: fullnameasstring
Comparing Truth Object: william lewis with Generated Object: william lewis reece


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Comparing Truth Predicate: hasname with Generated Predicate: hassurnameasstring
Comparing Truth Object: william lewis with Generated Object: reece


...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Comparing Truth Predicate: instance of with Generated Predicate: has_race
Comparing Truth Object: person with Generated Object: african-american


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Comparing Truth Predicate: instance of with Generated Predicate: hasagevalue
Comparing Truth Object: person with Generated Object: nan


TypeError: object of type 'float' has no len()